#Imports

In [ ]:
!pip install pulp

In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, value
import numpy as np

#Dataset Creation

In [ ]:
# Simulated data for EVs, Charging Stations, and Routes
num_evs = 7
num_stations = 3

ev_df = pd.DataFrame({'EV_ID': range(1, num_evs + 1),
                      'Starting_Energy': np.random.randint(50, 100, num_evs)})
ev_df['Required_energy'] = np.random.randint(80, 120, len(ev_df))

charging_station_df = pd.DataFrame({'Charging_Station_ID': range(1, num_stations + 1),
                                   'Charging_Rate': np.random.uniform(0.1, 0.5, num_stations),
                                   'Waiting_Time': np.random.randint(5, 15, num_stations)})

route_data = []
for ev in range(1, num_evs + 1):
    for station in range(1, num_stations + 1):
        route_data.append({
            'EV_ID': ev,
            'Charging_Station_ID': station,
            'Route_Time': np.random.randint(10, 60),
            'Total_Energy_Consumption': np.random.randint(20, 40)  # Total energy consumption for the route
        })

route_df = pd.DataFrame(route_data)

In [ ]:
ev_df

,EV_ID,Starting_Energy,Required_energy
0,1,79,101
1,2,53,95
2,3,98,94
3,4,54,115
4,5,57,119
5,6,99,100
6,7,81,106


In [ ]:
charging_station_df

,Charging_Station_ID,Charging_Rate,Waiting_Time
0,1,0.183659,10
1,2,0.273708,7
2,3,0.191717,12


In [ ]:
route_df

,EV_ID,Charging_Station_ID,Route_Time,Total_Energy_Consumption
0,1,1,13,23
1,1,2,52,21
2,1,3,25,30
3,2,1,53,25
4,2,2,27,28
5,2,3,22,30
6,3,1,16,29
7,3,2,52,25
8,3,3,11,27
9,4,1,23,37


In [ ]:
import random

#LP Solver to solve the problem using PuLP Library

In [ ]:
# # Create the LP problem
# model = LpProblem("EV_Assignment", sense=1)  # Minimization problem

# evs = ev_df['EV_ID'].tolist()
# stations = charging_station_df['Charging_Station_ID'].tolist()

# x = LpVariable.dicts("Assignment", (evs, stations), cat='Binary')

# # Objective function: minimize the maximum completion time
# # model += lpSum(x[ev][station] * ((route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Route_Time'].values[0]
# #                 + route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Waiting_Time'].values[0])
# #                 / charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0])
# #                 for ev in evs for station in stations)

# # Objective function: Minimize the maximum completion time among all stations
# max_completion_time = LpVariable("Max_Completion_Time", lowBound=0)

# for station in stations:
#     m_values = [(max(route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Route_Time'].values[0],charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Waiting_Time'].values[0])
#         + (ev_df.loc[(ev_df['EV_ID'] == ev), 'Required_energy'].values[0] - ev_df.loc[ev_df['EV_ID'] == ev, 'Starting_Energy'].values[0] + route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Total_Energy_Consumption'].values[0])
#         / charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0]
#     ) for ev in evs]

#     # Calculate the completion time of the station
#     ws = charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Waiting_Time'].values[0]
#     completion_time = lpSum(m_values-ws) + ws
#     # Update the objective function to minimize the maximum completion time
#     model+= max_completion_time >= completion_time

# # # Minimize the maximum completion time
# # model += max_completion_time
# # print(model)
# # Constraints: Assign each EV to exactly one station and energy consumption constraint
# for ev in evs:
#     model += lpSum(x[ev][station] for station in stations) == 1  # Each EV is assigned to exactly one station

#     for station in stations:
#         route_info = route_df[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station)]
#         if not route_info.empty:
#             start_energy = ev_df.loc[ev_df['EV_ID'] == ev, 'Starting_Energy'].values[0]
#             total_consumption = route_info['Total_Energy_Consumption'].values[0]
#             # energy_needed = total_consumption * charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0]

#             # Ensure the EV's starting energy is sufficient for the route
#             if start_energy < total_consumption:
#                 # If starting energy is less than energy needed for the route, do not assign the EV to this station
#                 x[ev][station] = 0

# # Solve the problem
# model.solve()
# # Output: Assignment and Maximum Completion Time
# best_time = value(model.objective)
# best_assignment = {(ev, station): x[ev][station].varValue for ev in evs for station in stations}
# # print(best_assignment)
# print("Best Assignment:")
# for (ev, station), assignment in best_assignment.items():
#     if assignment > 0:
#         print(f"EV {ev} assigned to Charging Station {station}")

# print("Minimum Maximum Completion Time:", best_time)


In [ ]:
# Create the LP problem
model = LpProblem("EV_Assignment", sense=1)  # Minimization problem

evs = ev_df['EV_ID'].tolist()
stations = charging_station_df['Charging_Station_ID'].tolist()
# paths = route_df['Path_id'].tolist()

x = LpVariable.dicts("Assignment", (evs, stations), cat='Binary')

# for path in paths:
#   print(path)
#   route_df['Cpes'][path] = max(route_df['Route_time'][path],charging_station_df.loc[charging_station_df['Charging_Station_ID'] == route_df['Charging_Station_ID'][path], 'Waiting_Time'].values[0]) +
#   ((ev_df.loc[(ev_df['EV_ID'] == route_df['EV_ID'][path]), 'Required_energy'].values[0] - ev_df.loc[ev_df['EV_ID'] == route_df['EV_ID'][path], 'Starting_Energy'].values[0] + route_df['Total_Energy_Consumption'][path])/ charging_station_df.loc[charging_station_df['Charging_Station_ID'] == route_df['Charging_Station_ID'][path], 'Charging_Rate'].values[0])

# Objective function: minimize the maximum completion time
# model += lpSum(x[ev][station] * ((route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Route_Time'].values[0]
#                 + route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Waiting_Time'].values[0])
#                 / charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0])
#                 for ev in evs for station in stations)

# Objective function: Minimize the maximum completion time among all stations
max_completion_time = LpVariable("Max_Completion_Time", lowBound=0)

for station in stations:
    m_values = [(max(route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Route_Time'].values[0],charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Waiting_Time'].values[0])
        + (ev_df.loc[(ev_df['EV_ID'] == ev), 'Required_energy'].values[0] - ev_df.loc[ev_df['EV_ID'] == ev, 'Starting_Energy'].values[0] + route_df.loc[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station), 'Total_Energy_Consumption'].values[0])
        / charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0]
    ) for ev in evs]

    # Calculate the completion time of the station
    ws = charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Waiting_Time'].values[0]
    completion_time = lpSum(m_values-ws) + ws
    # Update the objective function to minimize the maximum completion time
    model+= max_completion_time >= completion_time

# Minimize the maximum completion time
model += max_completion_time
# print(model)
# Constraints: Assign each EV to exactly one station and energy consumption constraint
for ev in evs:
    model += lpSum(x[ev][station] for station in stations) == 1  # Each EV is assigned to exactly one station

    for station in stations:
        route_info = route_df[(route_df['EV_ID'] == ev) & (route_df['Charging_Station_ID'] == station)]
        if not route_info.empty:
            start_energy = ev_df.loc[ev_df['EV_ID'] == ev, 'Starting_Energy'].values[0]
            total_consumption = route_info['Total_Energy_Consumption'].values[0]
            # energy_needed = total_consumption * charging_station_df.loc[charging_station_df['Charging_Station_ID'] == station, 'Charging_Rate'].values[0]

            # Ensure the EV's starting energy is sufficient for the route
            if start_energy < total_consumption:
                # If starting energy is less than energy needed for the route, do not assign the EV to this station
                x[ev][station] = 0

# Solve the problem
model.solve()
# Output: Assignment and Maximum Completion Time
best_time = value(model.objective)
best_assignment = {(ev, station): x[ev][station].varValue for ev in evs for station in stations}
# print(best_assignment)
print("Best Assignment:")
for (ev, station), assignment in best_assignment.items():
    if assignment > 0:
        print(f"EV {ev} assigned to Charging Station {station}")

print("Minimum Maximum Completion Time:", best_time)


Best Assignment:
EV 1 assigned to Charging Station 1
EV 2 assigned to Charging Station 1
EV 3 assigned to Charging Station 3
EV 4 assigned to Charging Station 3
EV 5 assigned to Charging Station 1
EV 6 assigned to Charging Station 3
EV 7 assigned to Charging Station 2
Minimum Maximum Completion Time: 2422.6231


In [ ]:
# first uniform out all the data
# then precaluclate cpes for each path
# then write the objective function as cpes * x[ev][station] for each station with each ev and then write the maximum.